# Lecture 2
## Linear Algebra

-----

### Intro

In [55]:
import numpy as np
import scipy.linalg
np.random.seed(1)

"""
Equation:
    3x + 6y + 5z = 30
    4x - 2y + 1z =  3
    2x + 4y - 3z =  1
    
    [3  6  5][x]   [30]
    [4 -2  1][y] = [ 3]
    [2  4 -3][z]   [ 1]
       ↑      ↑     ↑
       A      x  =  b
"""

A = [[3,  6,  5],
     [4, -2,  1],
     [2,  4, -3]]

b = [30, 3, 1]

print("[x, y, z] = ", np.linalg.solve(A, b))


[x, y, z] =  [1. 2. 3.]


### Gauss elimination (row reduction)

In [56]:
combined = np.c_[A, b]
print(scipy.linalg.lu(combined)[-1])

[[  4.          -2.           1.           3.        ]
 [  0.           7.5          4.25        27.75      ]
 [  0.           0.          -6.33333333 -19.        ]]


### Multiplication, Identity, and Inverse

Multiplication is **not** commutative. The multiplicative identity $I$ satisfies $A = AI = IA$, it's essentially the number "1" but for matrices. The inverse, $A^{-1}$, of $A$ satisfies $A^{-1}A = AA^{-1} = I$. In other words, right/left multiplication by the inverse is like right/left division.

In [57]:
A = [[1, 2],
     [3, 4]]
B = [[2, 3],
     [5, 7]]

A = np.array(A)
B = np.array(B)
A_inv = np.linalg.inv(A)


with np.printoptions(precision=3, suppress=True):
    print("AB =\n", A @ B, "\n")
    print("BA =\n", B @ A, "\n")
    print("A⁻¹ =\n", A_inv, "\n")
    print("A⁻¹A = AA⁻¹ =\n", A_inv @ A)

AB =
 [[12 17]
 [26 37]] 

BA =
 [[11 16]
 [26 38]] 

A⁻¹ =
 [[-2.   1. ]
 [ 1.5 -0.5]] 

A⁻¹A = AA⁻¹ =
 [[1. 0.]
 [0. 1.]]


### Jacobian + Newton's Method

Finding a root of
$$f(x) = \begin{bmatrix}1+x+y\\y-z\\ xy\end{bmatrix}$$

In [98]:
def f(x, y, z):
    return np.array([1 + x + y, y - z, x*y])

def Jacobian(x, y, z):
    return np.array([[1, 1,  0],
                     [0, 1, -1],
                     [y, x, 0]])

x, y, z = 1, 2, 3

print(f"x, y, z = {x}, {y}, {z}", "\n")
print("f(x, y, z) = ", f(x, y, z), "\n")
print("J(f) (x, y, z) = \n", Jacobian(x, y, z))

x, y, z = 1, 2, 3 

f(x, y, z) =  [ 4 -1  2] 

J(f) (x, y, z) = 
 [[ 1  1  0]
 [ 0  1 -1]
 [ 2  1  0]]


In [100]:
def Newton(f, threshold=1e-5):
    x, y, z = 1 + np.random.random(3) # Choose random 1 < x, y, z < 2
    while np.linalg.norm(f(x, y, z)) > threshold:
        J = Jacobian(x, y, z)
        dx, dy, dz = np.linalg.solve(J, f(x, y, z))
        x -= dx; y -= dy; z -= dz;
    return np.array([x, y, z])

with np.printoptions(precision=3, suppress=True):
    print("x, y, z = ", Newton(f))

x, y, z =  [ 0. -1. -1.]


### Broyden's Method

Approximates the Jacobian using previous iterations:
$$J_n\Delta x_n \approx \Delta f_n.$$
However, this isn't enough information to compute $J_n$.

-----
Example:
$$f = \begin{bmatrix}x+y\\x-y\end{bmatrix},$$
and $$\Delta x = \Delta y = 1\implies \Delta f = \begin{bmatrix}2\\0\end{bmatrix}.$$
Both $$\begin{bmatrix}2&0\\0&0\end{bmatrix},\text{ and }\begin{bmatrix}1&1\\1&-1\end{bmatrix}$$
could be $J$.

-----
Use additional condition: For any vector $y$ perpendicular to $\Delta x_n$, we force $J_ny = J_{n-1}y,$ which means we're only modifying $J$ in directions we have new information on. This gives
$$J_n = J_{n-1} + (\dots)\Delta x_n^T,$$
and plugging back into the first condition yields
$$J_n = J_{n-1} + \frac{\Delta f_n - J_{n-1}\Delta x_n}{\Delta x_n^T \Delta x_n}\Delta x_n^T$$

In [117]:
def Broyden(f, threshold=1e-5):
    x, y, z = 1 + np.random.random(3) # Choose random 1 < x, y, z < 2
    J = 1 + np.random.random((3, 3)) # Choose random initial Jacobian.
    J = np.identity(3)
    f_curr = f(x, y, z)
    while np.linalg.norm(f(x, y, z)) > threshold:
        f_prev, f_curr = f_curr, f(x, y, z)
        dx, dy, dz = np.linalg.solve(J, f_curr)
        x -= dx; y -= dy; z -= dz;
        
        delta = -np.array([dx, dy, dz])
        df = f_curr - f_prev
        print(J @ delta, df, delta)
        J += np.outer((df - J @ delta) / (delta @ delta), delta.T)
        print(J @ delta)
    return np.array([x, y, z])

with np.printoptions(precision=3, suppress=True):
    print("x, y, z = ", Broyden(f))

[-3.9   -0.107 -2.001] [0. 0. 0.] [-3.9   -0.107 -2.001]
[-0.  0. -0.]
[-0.755 -1.964  2.08 ] [-4.008  1.893 -4.18 ] [1.370e+16 3.772e+14 7.027e+15]
[-4.035  1.887 -4.481]
[ 2.846e+27 -4.828e+27 -4.927e+30] [ 1.408e+16 -6.649e+15  5.167e+30] [5.476e+45 1.508e+44 2.809e+45]
[ 2.846e+27 -4.828e+27  5.179e+30]
[ 2.067e+88 -1.167e+87 -8.428e+89] [ 5.627e+45 -2.658e+45  8.258e+89] [-8.750e+104 -2.410e+103 -4.489e+104]
[-1.116e+88 -1.167e+87  7.792e+89]
[-1.144e+207  9.944e+205 -2.026e+208] [-8.991e+104  4.248e+104  2.109e+208] [2.532e+223 6.974e+221 1.299e+223]
[-1.144e+207  9.944e+205 -2.026e+208]
[nan nan nan] [ 2.602e+223 -1.229e+223         inf] [nan nan nan]
[nan nan nan]
x, y, z =  [nan nan nan]


C:\Users\James Camacho\AppData\Local\Temp\ipykernel_11268\3386974444.py:14: RuntimeWarning: overflow encountered in matmul
  J += np.outer((df - J @ delta) / (delta @ delta), delta.T)
C:\Users\James Camacho\AppData\Local\Temp\ipykernel_11268\3103511659.py:2: RuntimeWarning: overflow encountered in scalar multiply
  return np.array([1 + x + y, y - z, x*y])
